In [1]:
import json
import random
import torch

In [2]:
!pip3 install sentencepiece

In [3]:
import pandas as pd

dataset = pd.read_csv("data_full_v1.csv")

In [4]:
dataset['ground_truth'] = dataset.apply(lambda row: 1 if row['label']=='yes' else 0, axis=1)

dataset['prompt'] = dataset.apply(lambda row: (row['prompt']).removeprefix('Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships: '), axis=1)


In [5]:
dataset['prompt'] = dataset['prompt'].str.lower()

In [6]:
dataset

,id,prompt,label,ground_truth
0,4,husband has a direct effect on wife and alarm ...,yes,1
1,7,husband has a direct effect on wife and alarm ...,no,0
2,8,husband has a direct effect on wife and alarm ...,yes,1
3,15,husband has a direct effect on wife and alarm ...,no,0
4,21,husband has a direct effect on wife and alarm ...,no,0
...,...,...,...,...
10107,31012,zuph has a direct effect on glimx. jyka has a ...,yes,1
10108,31014,zuph has a direct effect on glimx. jyka has a ...,yes,1
10109,31015,zuph has a direct effect on glimx. jyka has a ...,no,0
10110,31016,zuph has a direct effect on glimx. jyka has a ...,no,0


In [7]:
dataset = json.loads(dataset.to_json(orient ='records'))

# Shuffle the dataset
random.shuffle(dataset)

# Define split sizes for a 75-25 split
train_size = int(0.75 * len(dataset))
valid_size = len(dataset) - train_size

# Split the dataset
train_data = dataset[:train_size]
validation_data = dataset[train_size:]

In [12]:
from torch.utils.data import Dataset

class CladderDataset(Dataset):
    def __init__(self, cladder_data, tokenizer):
        self.cladder_data = cladder_data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.cladder_data)

    def __getitem__(self, idx):
        data = self.cladder_data[idx]
        # print(data)
        prompt = data['prompt']
        label = data['ground_truth']

        # print(prompt)
        # print(label)

        # Tokenize
        encoded_input = self.tokenizer(prompt, padding='max_length', truncation=True, return_tensors='pt').to("cuda")
        # print(encoded_input)
        encoded_input['labels'] = torch.tensor(label).to("cuda")

        # print(encoded_input)

        return encoded_input


In [9]:
# from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
# import torch
# from torch.utils.data import DataLoader
# import torch.optim as optim

# # Load tokenizer and model
# tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
# model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels = 2)

In [10]:
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model.to(device)

In [13]:
# # Prepare the DataLoader
# cladder_dataset = CladderDataset(train_data, tokenizer)
# dataloader = DataLoader(cladder_dataset, batch_size=8, shuffle=True)

# # Optimizer and Loss Function
# optimizer = optim.Adam(model.parameters(), lr=1e-5)
# loss_fn = torch.nn.CrossEntropyLoss()

In [14]:
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
from transformers import XLMRobertaTokenizer as tkz
from transformers import XLMRobertaForSequenceClassification as mpc

# Load tokenizer and model
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 2)

# run = 0
# for run in range(1):

for run in range(3):
  model_base = 'xlm-roberta-base'
  tokenizer = tkz.from_pretrained(model_base)
  model = mpc.from_pretrained(model_base, num_labels = 2)

  device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
  model.to(device)

  # Prepare the DataLoader
  cladder_dataset = CladderDataset(train_data, tokenizer)
  dataloader = DataLoader(cladder_dataset, batch_size=8, shuffle=True)

  # Optimizer and Loss Function
  optimizer = optim.Adam(model.parameters(), lr=1e-5)
  loss_fn = torch.nn.CrossEntropyLoss()



  # Training Loop
  model.train()

  #change to 3

  for epoch in range(3):  # Number of epochs
    for batch in dataloader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].squeeze(1)
        attention_mask = batch['attention_mask'].squeeze(1)
        labels = batch['labels']

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        print(f"Epoch: {epoch}, Loss: {loss.item()}")


  model_path = 'cladder_' + model_base + '_' + str(run) + '.pt'
  torch.save(model.state_dict(), model_path)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 0, Loss: 0.7154527902603149
Epoch: 0, Loss: 0.5410580635070801
Epoch: 0, Loss: 0.6262006759643555
Epoch: 0, Loss: 0.6143247485160828
Epoch: 0, Loss: 0.6701018214225769
Epoch: 0, Loss: 0.6198359131813049
Epoch: 0, Loss: 0.8298303484916687
Epoch: 0, Loss: 0.6792395114898682
Epoch: 0, Loss: 0.689285159111023
Epoch: 0, Loss: 0.7877989411354065
Epoch: 0, Loss: 0.796680212020874
Epoch: 0, Loss: 0.717836320400238
Epoch: 0, Loss: 0.648492157459259
Epoch: 0, Loss: 0.6257838010787964
Epoch: 0, Loss: 0.7705484628677368
Epoch: 0, Loss: 0.604006290435791
Epoch: 0, Loss: 0.7794986963272095
Epoch: 0, Loss: 0.6271209120750427
Epoch: 0, Loss: 0.763855516910553
Epoch: 0, Loss: 0.721824049949646
Epoch: 0, Loss: 0.6433218121528625
Epoch: 0, Loss: 0.7914061546325684
Epoch: 0, Loss: 0.784032940864563
Epoch: 0, Loss: 0.5817656517028809
Epoch: 0, Loss: 0.7215293645858765
Epoch: 0, Loss: 0.738263726234436
Epoch: 0, Loss: 0.7282803654670715
Epoch: 0, Loss: 0.6682370901107788
Epoch: 0, Loss: 0.63295245170

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 0, Loss: 0.6837537288665771
Epoch: 0, Loss: 0.762065052986145
Epoch: 0, Loss: 0.7303719520568848
Epoch: 0, Loss: 0.6840200424194336
Epoch: 0, Loss: 0.6868753433227539
Epoch: 0, Loss: 0.6987009644508362
Epoch: 0, Loss: 0.7164541482925415
Epoch: 0, Loss: 0.7227944731712341
Epoch: 0, Loss: 0.7440455555915833
Epoch: 0, Loss: 0.7044734954833984
Epoch: 0, Loss: 0.6965540051460266
Epoch: 0, Loss: 0.7003310322761536
Epoch: 0, Loss: 0.6712517142295837
Epoch: 0, Loss: 0.7204933762550354
Epoch: 0, Loss: 0.6875738501548767
Epoch: 0, Loss: 0.7056829929351807
Epoch: 0, Loss: 0.6756588220596313
Epoch: 0, Loss: 0.7166956663131714
Epoch: 0, Loss: 0.7016079425811768
Epoch: 0, Loss: 0.6679626703262329
Epoch: 0, Loss: 0.6752448081970215
Epoch: 0, Loss: 0.6957448720932007
Epoch: 0, Loss: 0.712256669998169
Epoch: 0, Loss: 0.7779948115348816
Epoch: 0, Loss: 0.6744498014450073
Epoch: 0, Loss: 0.7008483409881592
Epoch: 0, Loss: 0.6511948108673096
Epoch: 0, Loss: 0.6648166179656982
Epoch: 0, Loss: 0.6896

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 0, Loss: 0.7009033560752869
Epoch: 0, Loss: 0.6717284917831421
Epoch: 0, Loss: 0.7149709463119507
Epoch: 0, Loss: 0.6845728754997253
Epoch: 0, Loss: 0.7288174629211426
Epoch: 0, Loss: 0.7174960970878601
Epoch: 0, Loss: 0.6842010021209717
Epoch: 0, Loss: 0.6875172853469849
Epoch: 0, Loss: 0.653256893157959
Epoch: 0, Loss: 0.7092187404632568
Epoch: 0, Loss: 0.6756414771080017
Epoch: 0, Loss: 0.6957301497459412
Epoch: 0, Loss: 0.7119998335838318
Epoch: 0, Loss: 0.7107303142547607
Epoch: 0, Loss: 0.6987756490707397
Epoch: 0, Loss: 0.6862319111824036
Epoch: 0, Loss: 0.7200286388397217
Epoch: 0, Loss: 0.7314010262489319
Epoch: 0, Loss: 0.7155773639678955
Epoch: 0, Loss: 0.7682124376296997
Epoch: 0, Loss: 0.6714560985565186
Epoch: 0, Loss: 0.6907401084899902
Epoch: 0, Loss: 0.6464331746101379
Epoch: 0, Loss: 0.7473073601722717
Epoch: 0, Loss: 0.7073944807052612
Epoch: 0, Loss: 0.6779633164405823
Epoch: 0, Loss: 0.7013996243476868
Epoch: 0, Loss: 0.6771862506866455
Epoch: 0, Loss: 0.680

In [15]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

overall_accuracies, accuracies, f1s, precisions, recalls = [], [], [], [], []

# for run in range(1):
for run in range(3):
  model_path = 'cladder_' + model_base + '_' + str(run) + '.pt'
  print(model_path)

  model = mpc.from_pretrained(model_base, num_labels = 2)
  model.load_state_dict(torch.load(model_path))

  device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
  model.to(device)

  tokenizer = tkz.from_pretrained(model_base)
  validation_dataset = CladderDataset(validation_data, tokenizer)
  validation_dataloader = DataLoader(validation_dataset, batch_size=8)




  model.eval()
  correct_predictions = 0
  total_predictions = 0

  accuracy, f1, precision, recall = [], [], [], []
  batches = 0

  def compute_metrics(y_true, y_pred):
      accuracy = accuracy_score(y_true, y_pred)
      # print(y_true)
      # print(y_pred)
      f1 = f1_score(y_true, y_pred)

      precision = precision_score(y_true, y_pred)
      # print(precision)
      recall = recall_score(y_true, y_pred)
      return [accuracy, f1, precision, recall]

  with torch.no_grad():
      for batch in validation_dataloader:
          input_ids = batch['input_ids'].squeeze(1)
          attention_mask = batch['attention_mask'].squeeze(1)
          labels = batch['labels'].cpu()  # ground truth
          # print(labels)

          outputs = model(input_ids=input_ids, attention_mask=attention_mask)
          # print(outputs)
          # print(outputs.logits)
          predictions = torch.argmax(outputs.logits, dim=1).cpu()  # predictions
          # print(predictions)

          calculated_metrics = compute_metrics(labels, predictions)
          accuracy.append(calculated_metrics[0])
          f1.append(calculated_metrics[1])
          precision.append(calculated_metrics[2])
          recall.append(calculated_metrics[3])
          batches += 1

          correct_predictions += (predictions == labels).sum().item()
          total_predictions += labels.size(0)

  overall_accuracy = correct_predictions / total_predictions
  print(f"Overall Accuracy: {overall_accuracy:.4f}")
  overall_accuracies.append(overall_accuracy)

  accuracy_var = sum(accuracy)/batches
  print(f"Accuracy: {sum(accuracy)/batches:.4f}")
  print(accuracy_var)
  accuracies.append(accuracy_var)

  f1_var = sum(f1)/batches
  print(f"F1: {sum(f1)/batches:.4f}")
  print(f1_var)
  f1s.append(f1_var)

  precision_var = sum(precision)/batches
  print(f"Precision: {sum(precision)/batches:.4f}")
  print(precision_var)
  precisions.append(precision_var)

  recall_var = sum(recall)/batches
  print(f"Recall: {sum(recall)/batches:.4f}")
  print(recall_var)
  recalls.append(precision_var)

  print(f'Number of batches: {batches}')

cladder_xlm-roberta-base_0.pt


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Overall Accuracy: 0.4956
Accuracy: 0.4956
0.49564873417721517
F1: 0.6436
0.6435504720314849
Precision: 0.4956
0.49564873417721517
Recall: 0.9937
0.9936708860759493
Number of batches: 316
cladder_xlm-roberta-base_1.pt


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Overall Accuracy: 0.4956
Accuracy: 0.4956
0.49564873417721517
F1: 0.6436
0.6435504720314849
Precision: 0.4956
0.49564873417721517
Recall: 0.9937
0.9936708860759493
Number of batches: 316
cladder_xlm-roberta-base_2.pt


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Overall Accuracy: 0.4956
Accuracy: 0.4956
0.49564873417721517
F1: 0.6436
0.6435504720314849
Precision: 0.4956
0.49564873417721517
Recall: 0.9937
0.9936708860759493
Number of batches: 316


In [16]:
# # Training Loop
# model.train()

# for epoch in range(3):  # Number of epochs
#   for batch in dataloader:
#       optimizer.zero_grad()

#       input_ids = batch['input_ids'].squeeze(1)
#       attention_mask = batch['attention_mask'].squeeze(1)
#       labels = batch['labels']

#       outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
#       loss = outputs.loss

#       loss.backward()
#       optimizer.step()

#       print(f"Epoch: {epoch}, Loss: {loss.item()}")

In [17]:
# validation_dataset = CladderDataset(validation_data, tokenizer)
# validation_dataloader = DataLoader(validation_dataset, batch_size=8)

In [18]:
# model.eval()
# correct_predictions = 0
# total_predictions = 0

# with torch.no_grad():
#     for batch in validation_dataloader:
#         input_ids = batch['input_ids'].squeeze(1)
#         attention_mask = batch['attention_mask'].squeeze(1)
#         labels = batch['labels']

#         outputs = model(input_ids=input_ids, attention_mask=attention_mask)
#         predictions = torch.argmax(outputs.logits, dim=1)
#         correct_predictions += (predictions == labels).sum().item()
#         print(f"predictions: {predictions}, labels: {labels}")
#         total_predictions += labels.size(0)

# print(f"total_predictions: {total_predictions}")

# overall_accuracy = correct_predictions / total_predictions
# print(f"Overall Accuracy: {overall_accuracy:.4f}")
